In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from pandas.tseries.offsets import DateOffset

In [4]:

def merge_dataframes(df1, df2):
    """
    Merge the first 13 columns of two DataFrames.

    Parameters:
    df1 (DataFrame): First DataFrame.
    df2 (DataFrame): Second DataFrame.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame.

    Example:
        >>> df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
        >>> df2 = pd.DataFrame({'A': [7, 8, 9], 'C': [10, 11, 12]})
        >>> merge_dataframes(df1, df2)
           A  B   C
        0  1  4  10
        1  2  5  11
        2  3  6  12
    """
    # Determine the number of columns to merge (minimum between the two DataFrames)
    for col in df2.columns:
        if df2[col].dtype == 'datetime64[ns]':
            df2[col] = df2[col].dt.strftime('%Y-%m-%d')

    for col in df1.columns:
        if df1[col].dtype == 'datetime64[ns]':
            df1[col] = df1[col].dt.strftime('%Y-%m-%d')

    common_columns = df1.columns.intersection(df2.columns)

    merged_df = df1.merge(df2, on=common_columns.tolist(), how='outer')

    return merged_df

def merge_dataframes_with_previous_files(file_paths, column_patterns):
    """
    Merge the first 13 columns of DataFrames from multiple Excel files with two sheets each.
    The column names from the first DataFrame are kept.

    Parameters:
    file_paths (list): List of file paths to Excel files.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame from all files.
    """
    merged_df = None
    merged_df_temp = None
    
    # Iterate over each file path
    for file_path in file_paths:
        print(f"\nLoading file : {file_path}")
        # Load Excel file into separate DataFrames
        # Fetch only first sheet as it is RES 
        df = pd.read_excel(file_path, header=1, sheet_name=[0], parse_dates=False)
        print(f"Number of sheets detected: {len(df.keys())}")

        # Merge in sheet level
        for sheet_name, df_sheet in df.items():
            # Filter according to pattern columns to keep
            df_sheet = df_sheet.filter(regex='|'.join(column_patterns))
            df_sheet['file'] = file_path
            print(f"Number of columns detected in {sheet_name} : {len(df_sheet.columns)}")
            if merged_df_temp is None:
                # Merge the Sheet DataFrames
                merged_df_temp = df_sheet
            else:
                merged_df_temp = merge_dataframes(merged_df_temp, df_sheet)

        # Merge in file level
        # If merged_df is None, assign it the merged DataFrame, else merge with the previous merged DataFrame
        if merged_df is None:
            merged_df = merged_df_temp
        else:
            merged_df = merge_dataframes(merged_df, merged_df_temp)

    return merged_df

def fillna_column(df, column_name, fillna_column_name):
    """
    Fill missing values in a column by using values from another column.

    Args:
        df (pandas.DataFrame): The DataFrame containing the columns.
        column_name (str): The name of the column to fill missing values.
        fillna_column_name (str): The name of the column to use for filling missing values.

    Returns:
        pandas.DataFrame: The DataFrame with missing values filled in the specified column.
    """
    df[column_name] = df[column_name].fillna(df[fillna_column_name])
    df.drop(fillna_column_name, axis=1, inplace=True)
    return df

def convert_to_datetime(df, columns=None):
    """
    Convert date-like columns in a DataFrame to datetime type.

    Parameters:
    df (DataFrame): Input DataFrame.
    columns (list of str, optional): List of columns to convert. 
                                     If None, attempts to convert all object type columns.

    Returns:
    DataFrame: DataFrame with date-like columns converted to datetime type.
    """
    # Copy the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # If no specific columns are provided, select all object type columns
    if columns is None:
        columns = df_copy.columns[df_copy.dtypes == 'object']
    
    # Iterate through each specified column
    for col in columns:
        print(f"Attempting column: {col}")
        # Check if the column can be converted to datetime
        try:
            df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')
            print(f"Column {col} transformed successfully.\n")
        except Exception as e:
            print(f"Column {col} is incompatible with datetime: {e}\n")
            # If conversion fails, skip to the next column
            continue
            
    return df_copy

### Licenses Fetch

In [5]:

# Define the file paths and column patterns
# Define the directory containing Excel files
directory = "../../data/licenses/"
# Print the current working directory
print(f"Current working directory: {os.getcwd()}")
# Get the directory of the current file
current_directory = os.getcwd()
directory = os.path.join(current_directory, directory)
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".xlsx")]
column_patterns = ['ΑΙΤΗΣΗ', 'ΜΗΤΡΩΟ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ.*', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡ.*ΕΚΔ.*ΠΑΡΑΓ.*', 'ΗΜΕΡ.*ΛΗΞΗ.*ΠΑΡΑΓ.*', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '.*(MW)', 'ΤΕΧΝΟΛ.*'] 

# Sort file paths based on their names to process them in order
file_paths.sort()

licenses_df = merge_dataframes_with_previous_files(file_paths, column_patterns)

Current working directory: /home/marios/projects/RAE_Forecasting/src/notebooks

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/Vevaioseis-APE.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-OKTΩΒΡΙΟΣ-2023.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΙΑΝΟΥΑΡΙΟΣ-2023.xlsx


/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΙΟΥΛΙΟΣ-2023.xlsx


/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΙΟΥΝΙΟΣ-2023.xlsx


...

/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path



Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/Ν.4685-ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΟΚΤΩΒΡΙΟΣ-2021.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14


/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path



Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/Ν.4685-ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΣΕΠΤΕΜΒΡΙΟΣ-2022.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/Ν.4685_1.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/licenses/ΠΙΝΑΚΑΣ-ΑΙΤΙΟΛΟΓΗΜΕΝΩΝ-ΕΝΣΤΑΣΕΩΝ_ΑΝΑΡΤΗΣΗ_ΤΕΛΙΚΟ.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 9


/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].dt.strftime('%Y-%m-%d')


In [6]:
licenses_df

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file,AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ,ΗΜΕΡ/ΝΙΑ ΕΚΔ. ΑΔ. ΠΑΡΑΓΩΓΗΣ,ΙΣΧΥΣ (MW)
0,514,2001-02-19 00:00:00,ΛΑΚΜΟΣ ΕΝΕΡΓΕΙΑΚΗ ΑΕ,ΑΔ-00001,2001-05-29 00:00:00,2026-05-29 00:00:00,ΗΠΕΙΡΟΥ,ΙΩΑΝΝΙΝΩΝ,ΜΕΤΣΟΒΟΥ,ΕΓΝΑΤΙΑΣ,ΓΚΟΥΡΑ ΜΙΚΡΟ ΠΕΡΙΣΤΕΡΙ,2.155,ΜΥΗΕ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
1,67,2001-02-13 00:00:00,ΑΡΑΜΠΑΤΖΗΣ Β.Γ. ΑΕ,ΑΔ-00002,2001-05-29 00:00:00,2026-05-29 00:00:00,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,ΑΛΜΩΠΙΑΣ,ΑΡΙΔΑΙΑΣ,ΤΟΥΠΛΙΤΣΑ ΠΡΟΜΑΧΩΝΑ,0.830,ΜΥΗΕ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
2,38,2001-02-08 00:00:00,ΚΑΣΤΡΙ ΕΥΒΟΙΑΣ ΑΕ,ΑΔ-00022,2001-06-18 00:00:00,2041-03-22 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ,ΚΑΣΤΡΙ,5.000,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
3,439,2001-02-19 00:00:00,ΥΔΡΟΗΛΕΚΤΡΙΚΗ ΕΥΡΥΤΑΝΙΑΣ ΑΕ,ΑΔ-00023,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΡΥΤΑΝΙΑΣ,ΑΓΡΑΦΩΝ,ΑΓΡΑΦΩΝ,ΜΟΝΑΣΤΗΡΑΚΙ,3.000,ΜΥΗΕ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
4,65,2001-02-12 00:00:00,ΠΟΛΥΠΟΤΑΜΟΣ ΑΙΟΛΙΚΗ ΕΝΕΡΓΕΙΑ ΑΕ,ΑΔ-00024,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΓΚΕΡΚΙ-ΠΥΡΓΑΡΙ ΠΟΛΥΠΟΤΑΜΟΥ,12.000,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191143,NaN,NaN,GREEN VELOCITY 2,NaN,NaN,NaN,ΘΕΣΣΑΛΙΑΣ,ΤΡΙΚΑΛΩΝ,NaN,"ΑΙΘΗΚΩΝ,ΑΣΠΡΟΠΟΤΑΜΟΥ",ΝΕΡΑΙΔΑ,NaN,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,2020-09-26,14.4
2191144,NaN,NaN,GREEN VELOCITY 2,NaN,NaN,NaN,ΘΕΣΣΑΛΙΑΣ,ΤΡΙΚΑΛΩΝ,NaN,"ΑΣΠΡΟΠΟΤΑΜΟΥ,ΚΛΕΙΝΟΒΟΥ",ΚΑΛΤΣΑ,NaN,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,2020-09-26,14.4
2191145,NaN,NaN,WIND ENERGY FACILITY ΜΟΝΟΠΡΟΣΩΠΗ ΙΔΙΩΤΙΚΗ ΚΕΦΑ...,NaN,NaN,NaN,ΑΝ ΜΑΚΕΔΟΝΙΑΣ ΘΡΑΚΗΣ,ΡΟΔΟΠΗΣ,NaN,ΦΙΛΛΥΡΑΣ,ΚΑΤΩ ΒΡΑΧΟΣ,NaN,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,2020-09-28,3.0
2191146,NaN,NaN,WIND ENERGY FACILITY ΜΟΝΟΠΡΟΣΩΠΗ ΙΔΙΩΤΙΚΗ ΚΕΦΑ...,NaN,NaN,NaN,Δ ΜΑΚΕΔΟΝΙΑΣ,ΚΟΖΑΝΗΣ,NaN,ΒΛΑΣΤΗΣ,ΜΠΕΚΡΕΒΙΝΙΚΟΣ,NaN,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,2020-09-28,3.0


In [7]:
# Define the file paths and column patterns
# Define the directory containing Excel files
directory = "../../data/special_projects/"
# Print the current working directory
print(f"Current working directory: {os.getcwd()}")
# Get the directory of the current file
current_directory = os.getcwd()
directory = os.path.join(current_directory, directory)
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".xlsx")]
column_patterns = ['ΑΙΤΗΣΗ', 'ΜΗΤΡΩΟ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ.*', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡ.*ΕΚΔ.*ΠΑΡΑΓ.*', 'ΗΜΕΡ.*ΛΗΞΗ.*ΠΑΡΑΓ.*', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '.*(MW)', 'ΤΕΧΝΟΛ.*'] 

# Sort file paths based on their names to process them in order
file_paths.sort()

special_proj_df = merge_dataframes_with_previous_files(file_paths, column_patterns)

Current working directory: /home/marios/projects/RAE_Forecasting/src/notebooks

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202012.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202103.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202105.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202108.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202110.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasti

/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202303.xlsx


/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202305.xlsx


...

Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202307.xlsx


/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202309.xlsx


/tmp/ipykernel_8225/675629782.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_8225/675629782.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202310.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202311.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202402.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : /home/marios/projects/RAE_Forecasting/src/notebooks/../../data/special_projects/202403.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14


In [8]:
orig_licenses_df = licenses_df.copy()

In [9]:
licenses_df.head()

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file,AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ,ΗΜΕΡ/ΝΙΑ ΕΚΔ. ΑΔ. ΠΑΡΑΓΩΓΗΣ,ΙΣΧΥΣ (MW)
0,514,2001-02-19 00:00:00,ΛΑΚΜΟΣ ΕΝΕΡΓΕΙΑΚΗ ΑΕ,ΑΔ-00001,2001-05-29 00:00:00,2026-05-29 00:00:00,ΗΠΕΙΡΟΥ,ΙΩΑΝΝΙΝΩΝ,ΜΕΤΣΟΒΟΥ,ΕΓΝΑΤΙΑΣ,ΓΚΟΥΡΑ ΜΙΚΡΟ ΠΕΡΙΣΤΕΡΙ,2.155,ΜΥΗΕ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
1,67,2001-02-13 00:00:00,ΑΡΑΜΠΑΤΖΗΣ Β.Γ. ΑΕ,ΑΔ-00002,2001-05-29 00:00:00,2026-05-29 00:00:00,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,ΑΛΜΩΠΙΑΣ,ΑΡΙΔΑΙΑΣ,ΤΟΥΠΛΙΤΣΑ ΠΡΟΜΑΧΩΝΑ,0.830,ΜΥΗΕ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
2,38,2001-02-08 00:00:00,ΚΑΣΤΡΙ ΕΥΒΟΙΑΣ ΑΕ,ΑΔ-00022,2001-06-18 00:00:00,2041-03-22 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ,ΚΑΣΤΡΙ,5.000,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
3,439,2001-02-19 00:00:00,ΥΔΡΟΗΛΕΚΤΡΙΚΗ ΕΥΡΥΤΑΝΙΑΣ ΑΕ,ΑΔ-00023,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΡΥΤΑΝΙΑΣ,ΑΓΡΑΦΩΝ,ΑΓΡΑΦΩΝ,ΜΟΝΑΣΤΗΡΑΚΙ,3.000,ΜΥΗΕ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN
4,65,2001-02-12 00:00:00,ΠΟΛΥΠΟΤΑΜΟΣ ΑΙΟΛΙΚΗ ΕΝΕΡΓΕΙΑ ΑΕ,ΑΔ-00024,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΓΚΕΡΚΙ-ΠΥΡΓΑΡΙ ΠΟΛΥΠΟΤΑΜΟΥ,12.000,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...,NaN,NaN,NaN


In [10]:
licenses_df = fillna_column(licenses_df, 'ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ', 'AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ')
licenses_df = fillna_column(licenses_df, 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΗΜΕΡ/ΝΙΑ ΕΚΔ. ΑΔ. ΠΑΡΑΓΩΓΗΣ')
licenses_df = fillna_column(licenses_df, 'ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW)', 'ΙΣΧΥΣ (MW)')

licenses_df.columns = special_proj_df.columns

In [11]:
licenses_df.isna().sum()

ΑΙΤΗΣΗ                             62
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ        79
ΕΤΑΙΡΕΙΑ                            4
ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ           6337
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ        4
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ      65
ΠΕΡΙΦΕΡΕΙΑ                          4
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ                4
ΔΗΜΟΣ                              65
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                   12
ΘΕΣΗ                                4
ΙΣΧΥΣ (MW)                          4
ΤΕΧΝΟΛΟΓΙΑ                          4
file                                0
dtype: int64

In [12]:
special_proj_df.head()

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,/home/marios/projects/RAE_Forecasting/src/note...
1,Γ-00798,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23,2021-03-23,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.0,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...
2,Γ-01329,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02,2021-10-02,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.8,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...
3,Γ-01089,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16,2021-10-16,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.0,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...
4,Γ-01328,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16,2034-06-16,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.8,ΑΙΟΛΙΚΑ,/home/marios/projects/RAE_Forecasting/src/note...


In [13]:
all_ape_data = pd.concat([licenses_df, special_proj_df], ignore_index=True)
all_ape_data['file'] = all_ape_data['file'].str.split('/').str[-1].str.split('.').str[0]

In [14]:
all_ape_data.isna().sum()

ΑΙΤΗΣΗ                             62
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ       101
ΕΤΑΙΡΕΙΑ                           26
ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ           6359
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ       26
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ      87
ΠΕΡΙΦΕΡΕΙΑ                         26
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ               26
ΔΗΜΟΣ                              87
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                   34
ΘΕΣΗ                               26
ΙΣΧΥΣ (MW)                         26
ΤΕΧΝΟΛΟΓΙΑ                         26
file                                0
dtype: int64

### Preprocessing

In [15]:
all_ape_data_nodup = all_ape_data.drop_duplicates()

In [16]:
all_ape_data_nodup

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file
0,514,2001-02-19 00:00:00,ΛΑΚΜΟΣ ΕΝΕΡΓΕΙΑΚΗ ΑΕ,ΑΔ-00001,2001-05-29 00:00:00,2026-05-29 00:00:00,ΗΠΕΙΡΟΥ,ΙΩΑΝΝΙΝΩΝ,ΜΕΤΣΟΒΟΥ,ΕΓΝΑΤΙΑΣ,ΓΚΟΥΡΑ ΜΙΚΡΟ ΠΕΡΙΣΤΕΡΙ,2.155,ΜΥΗΕ,Vevaioseis-APE
1,67,2001-02-13 00:00:00,ΑΡΑΜΠΑΤΖΗΣ Β.Γ. ΑΕ,ΑΔ-00002,2001-05-29 00:00:00,2026-05-29 00:00:00,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,ΑΛΜΩΠΙΑΣ,ΑΡΙΔΑΙΑΣ,ΤΟΥΠΛΙΤΣΑ ΠΡΟΜΑΧΩΝΑ,0.830,ΜΥΗΕ,Vevaioseis-APE
2,38,2001-02-08 00:00:00,ΚΑΣΤΡΙ ΕΥΒΟΙΑΣ ΑΕ,ΑΔ-00022,2001-06-18 00:00:00,2041-03-22 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ,ΚΑΣΤΡΙ,5.000,ΑΙΟΛΙΚΑ,Vevaioseis-APE
3,439,2001-02-19 00:00:00,ΥΔΡΟΗΛΕΚΤΡΙΚΗ ΕΥΡΥΤΑΝΙΑΣ ΑΕ,ΑΔ-00023,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΡΥΤΑΝΙΑΣ,ΑΓΡΑΦΩΝ,ΑΓΡΑΦΩΝ,ΜΟΝΑΣΤΗΡΑΚΙ,3.000,ΜΥΗΕ,Vevaioseis-APE
4,65,2001-02-12 00:00:00,ΠΟΛΥΠΟΤΑΜΟΣ ΑΙΟΛΙΚΗ ΕΝΕΡΓΕΙΑ ΑΕ,ΑΔ-00024,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΓΚΕΡΚΙ-ΠΥΡΓΑΡΙ ΠΟΛΥΠΟΤΑΜΟΥ,12.000,ΑΙΟΛΙΚΑ,Vevaioseis-APE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195112,Γ-011373_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04888,2022-11-04,2047-11-04,ΠΕΛΟΠΟΝΝΗΣΟΥ,ΑΡΚΑΔΙΑΣ,"ΜΕΓΑΛΟΠΟΛΗΣ,ΓΟΡΤΥΝΙΑΣ,ΤΡΙΠΟΛΗΣ","ΜΕΓΑΛΟΠΟΛΗΣ,ΤΡΙΚΟΛΩΝΩΝ,ΦΑΛΑΝΘΟΥ","ΜΠΛΕΣΙΒΟΣ, ΨΑΡΟΒΟΥΝΙ, ΛΙΜΝΕΣ ΡΟΔΙΑΣ, ΔΙΑΚΟΠΙ, ...",187.110,ΑΙΟΛΙΚΑ,202403
2195113,Γ-011380_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04889,2022-11-04,2047-11-04,"ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΠΕΛΟΠΟΝΝΗΣΟΥ","ΗΛΕΙΑΣ,ΑΡΚΑΔΙΑΣ,ΜΕΣΣΗΝΙΑΣ","ΑΝΔΡΙΤΣΑΙΝΑΣ - ΚΡΕΣΤΕΝΩΝ,ΜΕΓΑΛΟΠΟΛΗΣ,ΟΙΧΑΛΙΑΣ,...","ΑΛΙΦΕΙΡΑΣ,ΑΝΔΡΙΤΣΑΙΝΗΣ,ΓΟΡΤΥΝΟΣ,ΕΙΡΑΣ,ΖΑΧΑΡΩΣ,...","ΓΥΜΝΟΡΡΑΧΗ, ΡΟΣΚΙΟ, ΜΠΡΙΝΙΑ, ΜΠΑΝΤΑΒΑΣ, ΚΟΥΚΟΥ...",417.600,ΑΙΟΛΙΚΑ,202403
2195114,Γ-011383_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04890,2022-11-04,2047-11-04,"ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ","ΑΡΚΑΔΙΑΣ,ΑΧΑΪΑΣ,ΚΟΡΙΝΘΙΑΣ","ΓΟΡΤΥΝΙΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΤΡΙΠΟΛΗΣ,ΕΡΥΜΑΝΘΟΥ,ΣΙΚΥΩΝΙΩΝ","ΒΥΤΙΝΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΚΛΕΙΤΟΡΙΑΣ, ΚΛΕΙΤΟΡΟΣ,ΚΟΝ...","ΛΑΚΩΜΑΤΑ, ΠΑΝΑΓΙΑ, ΣΑΙΤΑΣ, ΔΡΑΚΟΒΟΥΝΙ, ΒΥΘΟΥΛΑ...",280.665,ΑΙΟΛΙΚΑ,202403
2195115,Γ-013272,2021-06-09,VOLTON ΕΛΛΗΝΙΚΗ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ,ΑΔ-04891,2022-11-04,2047-11-04,ΠΕΛΟΠΟΝΝΗΣΟΥ,"ΑΡΓΟΛΙΔΑΣ, ΑΡΚΑΔΙΑΣ","ΑΡΓΟΥΣ -ΜΥΚΗΝΩΝ, ΤΡΙΠΟΛΗΣ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ...","ΑΡΓΟΥΣ, ΒΑΛΤΕΤΣΙΟΥ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ΛΥΡΚΕΙΑ...",ΑΓΙΟΣ ΠΕΤΡΟΣ-ΒΑΛΤΕΤΣΙ-ΝΕΣΤΑΝΗ-ΤΣΕΜΠΕΡΟΥ,298.200,ΑΙΟΛΙΚΑ,202403


In [17]:
all_ape_data_nodup = convert_to_datetime(all_ape_data_nodup, columns=['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'])

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ transformed successfully.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed successfully.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed successfully.



In [18]:
all_ape_data_nodup

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file
0,514,2001-02-19,ΛΑΚΜΟΣ ΕΝΕΡΓΕΙΑΚΗ ΑΕ,ΑΔ-00001,2001-05-29,2026-05-29,ΗΠΕΙΡΟΥ,ΙΩΑΝΝΙΝΩΝ,ΜΕΤΣΟΒΟΥ,ΕΓΝΑΤΙΑΣ,ΓΚΟΥΡΑ ΜΙΚΡΟ ΠΕΡΙΣΤΕΡΙ,2.155,ΜΥΗΕ,Vevaioseis-APE
1,67,2001-02-13,ΑΡΑΜΠΑΤΖΗΣ Β.Γ. ΑΕ,ΑΔ-00002,2001-05-29,2026-05-29,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,ΑΛΜΩΠΙΑΣ,ΑΡΙΔΑΙΑΣ,ΤΟΥΠΛΙΤΣΑ ΠΡΟΜΑΧΩΝΑ,0.830,ΜΥΗΕ,Vevaioseis-APE
2,38,2001-02-08,ΚΑΣΤΡΙ ΕΥΒΟΙΑΣ ΑΕ,ΑΔ-00022,2001-06-18,2041-03-22,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ,ΚΑΣΤΡΙ,5.000,ΑΙΟΛΙΚΑ,Vevaioseis-APE
3,439,2001-02-19,ΥΔΡΟΗΛΕΚΤΡΙΚΗ ΕΥΡΥΤΑΝΙΑΣ ΑΕ,ΑΔ-00023,2001-06-18,2026-06-18,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΡΥΤΑΝΙΑΣ,ΑΓΡΑΦΩΝ,ΑΓΡΑΦΩΝ,ΜΟΝΑΣΤΗΡΑΚΙ,3.000,ΜΥΗΕ,Vevaioseis-APE
4,65,2001-02-12,ΠΟΛΥΠΟΤΑΜΟΣ ΑΙΟΛΙΚΗ ΕΝΕΡΓΕΙΑ ΑΕ,ΑΔ-00024,2001-06-18,2026-06-18,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΓΚΕΡΚΙ-ΠΥΡΓΑΡΙ ΠΟΛΥΠΟΤΑΜΟΥ,12.000,ΑΙΟΛΙΚΑ,Vevaioseis-APE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195112,Γ-011373_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04888,2022-11-04,2047-11-04,ΠΕΛΟΠΟΝΝΗΣΟΥ,ΑΡΚΑΔΙΑΣ,"ΜΕΓΑΛΟΠΟΛΗΣ,ΓΟΡΤΥΝΙΑΣ,ΤΡΙΠΟΛΗΣ","ΜΕΓΑΛΟΠΟΛΗΣ,ΤΡΙΚΟΛΩΝΩΝ,ΦΑΛΑΝΘΟΥ","ΜΠΛΕΣΙΒΟΣ, ΨΑΡΟΒΟΥΝΙ, ΛΙΜΝΕΣ ΡΟΔΙΑΣ, ΔΙΑΚΟΠΙ, ...",187.110,ΑΙΟΛΙΚΑ,202403
2195113,Γ-011380_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04889,2022-11-04,2047-11-04,"ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΠΕΛΟΠΟΝΝΗΣΟΥ","ΗΛΕΙΑΣ,ΑΡΚΑΔΙΑΣ,ΜΕΣΣΗΝΙΑΣ","ΑΝΔΡΙΤΣΑΙΝΑΣ - ΚΡΕΣΤΕΝΩΝ,ΜΕΓΑΛΟΠΟΛΗΣ,ΟΙΧΑΛΙΑΣ,...","ΑΛΙΦΕΙΡΑΣ,ΑΝΔΡΙΤΣΑΙΝΗΣ,ΓΟΡΤΥΝΟΣ,ΕΙΡΑΣ,ΖΑΧΑΡΩΣ,...","ΓΥΜΝΟΡΡΑΧΗ, ΡΟΣΚΙΟ, ΜΠΡΙΝΙΑ, ΜΠΑΝΤΑΒΑΣ, ΚΟΥΚΟΥ...",417.600,ΑΙΟΛΙΚΑ,202403
2195114,Γ-011383_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04890,2022-11-04,2047-11-04,"ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ","ΑΡΚΑΔΙΑΣ,ΑΧΑΪΑΣ,ΚΟΡΙΝΘΙΑΣ","ΓΟΡΤΥΝΙΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΤΡΙΠΟΛΗΣ,ΕΡΥΜΑΝΘΟΥ,ΣΙΚΥΩΝΙΩΝ","ΒΥΤΙΝΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΚΛΕΙΤΟΡΙΑΣ, ΚΛΕΙΤΟΡΟΣ,ΚΟΝ...","ΛΑΚΩΜΑΤΑ, ΠΑΝΑΓΙΑ, ΣΑΙΤΑΣ, ΔΡΑΚΟΒΟΥΝΙ, ΒΥΘΟΥΛΑ...",280.665,ΑΙΟΛΙΚΑ,202403
2195115,Γ-013272,2021-06-09,VOLTON ΕΛΛΗΝΙΚΗ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ,ΑΔ-04891,2022-11-04,2047-11-04,ΠΕΛΟΠΟΝΝΗΣΟΥ,"ΑΡΓΟΛΙΔΑΣ, ΑΡΚΑΔΙΑΣ","ΑΡΓΟΥΣ -ΜΥΚΗΝΩΝ, ΤΡΙΠΟΛΗΣ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ...","ΑΡΓΟΥΣ, ΒΑΛΤΕΤΣΙΟΥ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ΛΥΡΚΕΙΑ...",ΑΓΙΟΣ ΠΕΤΡΟΣ-ΒΑΛΤΕΤΣΙ-ΝΕΣΤΑΝΗ-ΤΣΕΜΠΕΡΟΥ,298.200,ΑΙΟΛΙΚΑ,202403


In [19]:
all_ape_data_nodup.isna().sum()

ΑΙΤΗΣΗ                             62
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ        99
ΕΤΑΙΡΕΙΑ                           25
ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ           5335
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ       44
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ     119
ΠΕΡΙΦΕΡΕΙΑ                         25
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ               25
ΔΗΜΟΣ                              86
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                   31
ΘΕΣΗ                               25
ΙΣΧΥΣ (MW)                         25
ΤΕΧΝΟΛΟΓΙΑ                         25
file                                0
dtype: int64

In [20]:
all_ape_data_nodup.columns

Index(['ΑΙΤΗΣΗ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΕΤΑΙΡΕΙΑ',
       'ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
       'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ',
       'ΔΗΜΟΣ ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', 'ΙΣΧΥΣ (MW)', 'ΤΕΧΝΟΛΟΓΙΑ',
       'file'],
      dtype='object')

In [40]:
df= all_ape_data_nodup.drop(columns=['ΕΤΑΙΡΕΙΑ','ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟΣ ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', 'ΤΕΧΝΟΛΟΓΙΑ',
'file']).copy()

# Convert date columns to datetime
date_columns = ['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ']


# Extend the DataFrame to include future dates
forecast_horizon = 360  # Number of days to forecast
# Convert the DataFrame index to datetime
df.index = pd.to_datetime(df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'])


# Now get the last_date
last_date = df.index[-1]

# Now this should work
future_dates = [last_date + pd.DateOffset(days=x) for x in range(1, forecast_horizon + 1)]
# Create a DataFrame for future dates
future_df = pd.DataFrame(index=future_dates, columns=df.columns)
df = pd.concat([df, future_df])

# df = df.sort_values(by='ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', ascending=True)
print(df.columns)

# Example of different aggregation functions for different columns
df = df.groupby(df.index).agg({
    'ΑΙΤΗΣΗ': 'count',  # Counting the number of entries per date
    'ΙΣΧΥΣ (MW)': ['sum', 'mean', 'max', 'min']  # Applying multiple aggregations to MW values
})

# # Flatten the column multi-index
# # df.columns = ['_'.join(col).strip() for col in df_aggregated.columns.values]
# df.rename(columns={
#     'ΑΙΤΗΣΗ_count': 'count_ΑΙΤΗΣΗ',
#     'ΙΣΧΥΣ (MW)_sum': 'total_ΙΣΧΥΣ (MW)',
#     'ΙΣΧΥΣ (MW)_mean': 'mean_ΙΣΧΥΣ (MW)',
#     'ΙΣΧΥΣ (MW)_max': 'max_ΙΣΧΥΣ (MW)',
#     'ΙΣΧΥΣ (MW)_min': 'min_ΙΣΧΥΣ (MW)'
# }, inplace=True)


print(df.columns)

# Extract date features
for col in date_columns:
    df[f'{col}_month'] = df[col].dt.month
    df[f'{col}_day'] = df[col].dt.day
    df[f'{col}_dayofweek'] = df[col].dt.dayofweek


days_to_lag = 200

for lag in range(0,days_to_lag):
    # Create lag features for the target variable
    df[f'ΙΣΧΥΣ (MW)_lag{lag}'] = df['ΙΣΧΥΣ (MW)'].shift(lag)

# Compute rolling statistics for the target variable
df['ΙΣΧΥΣ (MW)_rolling_mean'] = df['ΙΣΧΥΣ (MW)'].rolling(window=30).mean()
df['ΙΣΧΥΣ (MW)_rolling_std'] = df['ΙΣΧΥΣ (MW)'].rolling(window=30).std()

# Drop rows with NaN values created by lagging/rolling
df = df.dropna()

# Verify the DataFrame
print(df.info())
print(df.head())

Index(['ΑΙΤΗΣΗ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
       'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΙΣΧΥΣ (MW)'],
      dtype='object')
MultiIndex([(    'ΑΙΤΗΣΗ', 'count'),
            ('ΙΣΧΥΣ (MW)',   'sum'),
            ('ΙΣΧΥΣ (MW)',  'mean'),
            ('ΙΣΧΥΣ (MW)',   'max'),
            ('ΙΣΧΥΣ (MW)',   'min')],
           )


/tmp/ipykernel_8225/3464738961.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, future_df])


KeyError: 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ'

In [ ]:
df

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΙΣΧΥΣ (MW),ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ_month,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ_day,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ_dayofweek,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ_month,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ_day,...,ΙΣΧΥΣ (MW)_lag192,ΙΣΧΥΣ (MW)_lag193,ΙΣΧΥΣ (MW)_lag194,ΙΣΧΥΣ (MW)_lag195,ΙΣΧΥΣ (MW)_lag196,ΙΣΧΥΣ (MW)_lag197,ΙΣΧΥΣ (MW)_lag198,ΙΣΧΥΣ (MW)_lag199,ΙΣΧΥΣ (MW)_rolling_mean,ΙΣΧΥΣ (MW)_rolling_std
2002-07-18,Γ-00080,2001-11-16,2002-07-18,2027-07-18,0.997,11.0,16.0,4.0,7.0,18.0,...,0.30,1.30,11.22,12.00,3.00,5.00,0.830,2.155,6.773067,6.744491
2002-07-18,Γ-00027,2001-07-16,2002-07-18,2022-01-31,13.500,7.0,16.0,0.0,7.0,18.0,...,0.75,0.30,1.30,11.22,12.00,3.00,5.000,0.830,7.156400,6.790504
2002-07-18,Γ-00061,2001-10-25,2002-07-18,2027-07-18,1.500,10.0,25.0,3.0,7.0,18.0,...,1.00,0.75,0.30,1.30,11.22,12.00,3.000,5.000,6.866400,6.841622
2002-07-18,B-00184,2001-02-19,2002-07-18,2027-07-18,0.850,2.0,19.0,0.0,7.0,18.0,...,3.60,1.00,0.75,0.30,1.30,11.22,12.000,3.000,6.871400,6.837013
2002-07-18,Γ-00041,2001-09-07,2002-07-18,2027-07-18,44.000,9.0,7.0,4.0,7.0,18.0,...,5.00,3.60,1.00,0.75,0.30,1.30,11.220,12.000,7.768067,9.478444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-04,Γ-011373_Α,2020-12-16,2022-11-04,2047-11-04,187.110,12.0,16.0,2.0,11.0,4.0,...,45.00,8.00,20.70,37.80,79.80,298.20,280.665,417.600,60.600333,68.771478
2022-11-04,Γ-011380_Α,2020-12-16,2022-11-04,2047-11-04,417.600,12.0,16.0,2.0,11.0,4.0,...,13.80,45.00,8.00,20.70,37.80,79.80,298.200,280.665,73.773667,94.310449
2022-11-04,Γ-011383_Α,2020-12-16,2022-11-04,2047-11-04,280.665,12.0,16.0,2.0,11.0,4.0,...,24.00,13.80,45.00,8.00,20.70,37.80,79.800,298.200,77.795833,100.485558
2022-11-04,Γ-013272,2021-06-09,2022-11-04,2047-11-04,298.200,6.0,9.0,2.0,11.0,4.0,...,79.80,24.00,13.80,45.00,8.00,20.70,37.800,79.800,78.215833,101.406183


In [42]:
import pandas as pd

# Assuming 'all_ape_data_nodup' is your initial DataFrame
df = all_ape_data_nodup.drop(columns=[
    'ΕΤΑΙΡΕΙΑ', 'ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 
    'ΔΗΜΟΣ ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', 'ΤΕΧΝΟΛΟΓΙΑ', 'file'
]).copy()

# Convert date columns to datetime
date_columns = ['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Set the index to 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ' and sort
df.set_index('ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', inplace=True)
df.sort_index(inplace=True)

# Extract date features before aggregation
for col in date_columns:
    df[f'{col}_month'] = df[col].dt.month
    df[f'{col}_day'] = df[col].dt.day
    df[f'{col}_dayofweek'] = df[col].dt.dayofweek

# Aggregate the data by date
df_aggregated = df.groupby(df.index).agg({
    'ΑΙΤΗΣΗ': 'count',  # Counting the number of entries per date
    'ΙΣΧΥΣ (MW)': ['sum', 'mean', 'max', 'min']  # Applying multiple aggregations to MW values
})

# Flatten the column multi-index
df_aggregated.columns = ['_'.join(col).strip() for col in df_aggregated.columns.values]
df_aggregated.rename(columns={
    'ΑΙΤΗΣΗ_count': 'count_ΑΙΤΗΣΗ',
    'ΙΣΧΥΣ (MW)_sum': 'total_ΙΣΧΥΣ (MW)',
    'ΙΣΧΥΣ (MW)_mean': 'mean_ΙΣΧΥΣ (MW)',
    'ΙΣΧΥΣ (MW)_max': 'max_ΙΣΧΥΣ (MW)',
    'ΙΣΧΥΣ (MW)_min': 'min_ΙΣΧΥΣ (MW)'
}, inplace=True)

# Extend the DataFrame to include future dates
forecast_horizon = 360  # Number of days to forecast
last_date = df_aggregated.index[-1]
future_dates = [last_date + pd.DateOffset(days=x) for x in range(1, forecast_horizon + 1)]
future_df = pd.DataFrame(index=future_dates, columns=df_aggregated.columns)
df_aggregated = pd.concat([df_aggregated, future_df])

# Generate lag features
days_to_lag = 200
for lag in range(1, days_to_lag + 1):
    df_aggregated[f'ΙΣΧΥΣ (MW)_lag{lag}'] = df_aggregated['total_ΙΣΧΥΣ (MW)'].shift(lag)

# Compute rolling statistics
df_aggregated['ΙΣΧΥΣ (MW)_rolling_mean'] = df_aggregated['total_ΙΣΧΥΣ (MW)'].rolling(window=30).mean()
df_aggregated['ΙΣΧΥΣ (MW)_rolling_std'] = df_aggregated['total_ΙΣΧΥΣ (MW)'].rolling(window=30).std()

# Drop rows with NaN values created by lagging/rolling
df_aggregated.dropna(inplace=True)

# Verify the DataFrame
print(df_aggregated.info())
print(df_aggregated.head())


KeyError: 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'

In [23]:

# # Define the directory paths and column patterns
# directory_paths = ["../data/licenses/", "../data/contracts/"]
# column_patterns = ['ΑΙΤΗΣΗ', 'ΜΗΤΡΩΟ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ.*', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡ.*ΕΚΔ.*ΠΑΡΑΓ.*', 'ΗΜΕΡ.*ΛΗΞΗ.*ΠΑΡΑΓ.*', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '.*(MW)', 'ΤΕΧΝΟΛ.*'] 

# # Iterate over each directory path
# for directory_path in directory_paths:
#     # Print the current working directory
#     print(f"Current working directory: {os.getcwd()}")
#     # Get the directory of the current file
#     current_directory = os.path.dirname(os.path.abspath(__file__))
#     # Define the directory containing Excel files
#     directory = os.path.join(current_directory, directory_path)
#     file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".xlsx")]

# # Sort file paths based on their names to process them in order
# file_paths.sort()

# merged_df = merge_dataframes_with_previous_files(file_paths, column_patterns)
